# Retrain a VGG16 Architecture
* https://keras.io/applications/#vgg16
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
* https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
import numpy as np

In [4]:
from distutils.version import StrictVersion

In [5]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.3.0


In [7]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.8


In [8]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.1


## Preparation

In [9]:
INPUT_SHAPE = (64, 64)
# INPUT_SHAPE = (128, 128)
# INPUT_SHAPE = (256, 256)

In [10]:
EPOCHS = 50

In [11]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 100

In [12]:
!rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

In [13]:
!ls -lh

total 74G
-rwxrwxrwx 1 root root  44K Sep 28 19:12 440px-Beagle_Upsy.jpg
drwxrwxrwx 0 root root  512 Sep 30 12:14 augmented-signs
-rwxrwxrwx 1 root root  17M Sep 30 12:13 augmented-signs.zip
-rwxrwxrwx 1 root root 303K Sep 28 18:29 Black_New_York_stuy_town_squirrel_amanda_ernlund.jpeg
-rwxrwxrwx 1 root root 844K Sep 28 20:07 cat-bonkers.png
-rwxrwxrwx 1 root root 140K Sep 27 15:54 cnn-augmentation.ipynb
-rwxrwxrwx 1 root root 1.6M Sep 27 15:54 cnn-comparing-all-models.ipynb
-rwxrwxrwx 1 root root 102K Sep 30 13:50 cnn-imagenet-retrain.ipynb
-rwxrwxrwx 1 root root 158K Sep 30 10:48 cnn-intro.ipynb
-rwxrwxrwx 1 root root 1.3M Sep 28 21:22 cnn-prediction.ipynb
-rwxrwxrwx 1 root root 158K Sep 30 09:38 cnn-standard-architectures.ipynb
-rwxrwxrwx 1 root root 198K Sep 30 12:12 cnn-train-augmented.ipynb
-rwxrwxrwx 1 root root 495K Sep 28 18:29 london.jpg
drwxrwxrwx 0 root root  512 Sep 27 16:11 __MACOSX
-rwxrwxrwx 1 root root 127K Sep 28 18:29 Michigan-MSU-raschka.jpg
-rwxrwxrwx 1 root root 51

In [14]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [15]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
X, y = load_data(original_dir, type=".ppm")

### Uncomment next three cells if you want to train on augmented image set
#### Otherwise Overfitting can not be avoided because image set is simply too small

In [16]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip
# from zipfile import ZipFile
# zip = ZipFile('augmented-signs.zip')
# zip.extractall('.')

In [17]:
# data_dir = os.path.join(ROOT_PATH, "augmented-signs")
# augmented_images, augmented_labels = load_data(data_dir, type=".png")

In [18]:
# # merge both data sets

# all_images = np.vstack((X, augmented_images))
# all_labels = np.vstack((y, augmented_labels))

# # shuffle
# # https://stackoverflow.com/a/4602224

# p = numpy.random.permutation(len(all_labels))
# shuffled_images = all_images[p]
# shuffled_labels = all_labels[p]
# X, y = shuffled_images, shuffled_labels

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((303, 64, 64, 3), (303, 6))

## First Step: Load VGG pretrained on imagenet and remove classifier
### Hope: Feature Extraction will also work well for Speed Limit Signs

![VGG architecture](https://djcordhose.github.io/ai/img/sketch/vgg-no-classifier.png)

### Imagenet
* Collection of labelled images from many categories
* http://image-net.org/

http://image-net.org/about-stats

<table class="table-stats" style="width: 500px">
<tbody><tr>
<td width="25%"><b>High level category</b></td>
<td width="20%"><b># synset (subcategories)</b></td>
<td width="30%"><b>Avg # images per synset</b></td>
<td width="25%"><b>Total # images</b></td>
</tr>

<tr><td>amphibian</td><td>94</td><td>591</td><td>56K</td></tr>

<tr><td>animal</td><td>3822</td><td>732</td><td>2799K</td></tr>

<tr><td>appliance</td><td>51</td><td>1164</td><td>59K</td></tr>

<tr><td>bird</td><td>856</td><td>949</td><td>812K</td></tr>

<tr><td>covering</td><td>946</td><td>819</td><td>774K</td></tr>

<tr><td>device</td><td>2385</td><td>675</td><td>1610K</td></tr>

<tr><td>fabric</td><td>262</td><td>690</td><td>181K</td></tr>

<tr><td>fish</td><td>566</td><td>494</td><td>280K</td></tr>

<tr><td>flower</td><td>462</td><td>735</td><td>339K</td></tr>

<tr><td>food</td><td>1495</td><td>670</td><td>1001K</td></tr>

<tr><td>fruit</td><td>309</td><td>607</td><td>188K</td></tr>

<tr><td>fungus</td><td>303</td><td>453</td><td>137K</td></tr>

<tr><td>furniture</td><td>187</td><td>1043</td><td>195K</td></tr>

<tr><td>geological formation</td><td>151</td><td>838</td><td>127K</td></tr>

<tr><td>invertebrate</td><td>728</td><td>573</td><td>417K</td></tr>

<tr><td>mammal</td><td>1138</td><td>821</td><td>934K</td></tr>

<tr><td>musical instrument</td><td>157</td><td>891</td><td>140K</td></tr>


<tr><td>plant</td><td>1666</td><td>600</td><td>999K</td></tr>

<tr><td>reptile</td><td>268</td><td>707</td><td>190K</td></tr>

<tr><td>sport</td><td>166</td><td>1207</td><td>200K</td></tr>

<tr><td>structure</td><td>1239</td><td>763</td><td>946K</td></tr>

<tr><td>tool</td><td>316</td><td>551</td><td>174K</td></tr>

<tr><td>tree</td><td>993</td><td>568</td><td>564K</td></tr>

<tr><td>utensil</td><td>86</td><td>912</td><td>78K</td></tr>

<tr><td>vegetable</td><td>176</td><td>764</td><td>135K</td></tr>

<tr><td>vehicle</td><td>481</td><td>778</td><td>374K</td></tr>

<tr><td>person</td><td>2035</td><td>468</td><td>952K</td></tr>

</tbody></table>

### Might be more suitable for cats and dogs, but is the best we have right now

In [20]:
from keras import applications
# applications.VGG16?
vgg_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
# vgg_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

### All Convolutional Blocks are kept fully trained, we just removed the classifier part

In [21]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

### Next step is to push all our signs through the net just once and record the output of bottleneck features
#### Don't get confused: this is no training, yet, this just is recording the prediction in order not to repeat this expensive step over and over again when we train the classifier later

In [22]:
# will take a while, but not really long depending on size and number of input images

%time bottleneck_features_train = vgg_model.predict(X_train)

CPU times: user 1min 11s, sys: 9.16 s, total: 1min 20s
Wall time: 23.6 s


In [23]:
bottleneck_features_train.shape

(303, 2, 2, 512)

## What does this mean?
* 303 predictions for 303 images or 3335 predictions for 3335 images when using augmented data set
* 512 bottleneck feature per prediction
* each bottleneck feature has a size of 2x2, just a blob more or less
* bottleneck feature has larger size when we increase size of input images (might be a good idea)
  * 4x4 when using 128x128 as input

In [24]:
first_bottleneck_feature = bottleneck_features_train[0,:,:, 0]

In [25]:
first_bottleneck_feature

array([[ 0.01627164,  0.2276783 ],
       [ 0.        ,  0.        ]], dtype=float32)

## Now we create a new classifier and train it with this output and the labels from ground truth
### Classifier is copied from our first VGG style network

In [26]:
input_shape = bottleneck_features_train.shape[1:]

In [27]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input

# try and vary between .4 and .75
drop_out = 0.50

inputs = Input(shape=input_shape)

x = Flatten()(inputs)

# this is an additional dropout to compensate for the missing one after bottleneck features
x = Dropout(drop_out)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(drop_out)(x)

# softmax activation, 6 categories
predictions = Dense(6, activation='softmax')(x)

In [28]:
classifier_model = Model(input=inputs, output=predictions)
classifier_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 526,086
Trainable params: 526,086
Non-trainable params: 0
_________________________________________________________________


In [29]:
classifier_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
!rm -rf tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=/mnt/c/Users/olive/Development/ml/tf_log
# open http://localhost:6006

## This is a very simple architecture and should train pretty fast
* Depending on whether we used augmented data it either overfits quite a bit

In [31]:
%time history = classifier_model.fit(bottleneck_features_train, y_train, epochs=500, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])

Train on 242 samples, validate on 61 samples
Epoch 1/500
242/242 [==============================] - 0s - loss: 2.3052 - acc: 0.1777 - val_loss: 1.8253 - val_acc: 0.1967
Epoch 2/500
242/242 [==============================] - 0s - loss: 1.9942 - acc: 0.2521 - val_loss: 1.6149 - val_acc: 0.2295
Epoch 3/500
242/242 [==============================] - 0s - loss: 1.8625 - acc: 0.2975 - val_loss: 1.5362 - val_acc: 0.3279
Epoch 4/500
242/242 [==============================] - 0s - loss: 1.7044 - acc: 0.3264 - val_loss: 1.4821 - val_acc: 0.4426
Epoch 5/500
242/242 [==============================] - 0s - loss: 1.6068 - acc: 0.4091 - val_loss: 1.4879 - val_acc: 0.4098
Epoch 6/500
242/242 [==============================] - 0s - loss: 1.4906 - acc: 0.4132 - val_loss: 1.5074 - val_acc: 0.2295
Epoch 7/500
242/242 [==============================] - 0s - loss: 1.4711 - acc: 0.4421 - val_loss: 1.5089 - val_acc: 0.2295
Epoch 8/500
242/242 [==============================] - 0s - loss: 1.4265 - acc: 0.4132 

242/242 [==============================] - 0s - loss: 0.5077 - acc: 0.7893 - val_loss: 0.8818 - val_acc: 0.7869
Epoch 67/500
242/242 [==============================] - 0s - loss: 0.5253 - acc: 0.8182 - val_loss: 0.8927 - val_acc: 0.7541
Epoch 68/500
242/242 [==============================] - 0s - loss: 0.4585 - acc: 0.8347 - val_loss: 0.9045 - val_acc: 0.7705
Epoch 69/500
242/242 [==============================] - 0s - loss: 0.5393 - acc: 0.7934 - val_loss: 0.8907 - val_acc: 0.7869
Epoch 70/500
242/242 [==============================] - 0s - loss: 0.4690 - acc: 0.8140 - val_loss: 0.8937 - val_acc: 0.7705
Epoch 71/500
242/242 [==============================] - 0s - loss: 0.4764 - acc: 0.8182 - val_loss: 0.9035 - val_acc: 0.7377
Epoch 72/500
242/242 [==============================] - 0s - loss: 0.4389 - acc: 0.8140 - val_loss: 0.9302 - val_acc: 0.7705
Epoch 73/500
242/242 [==============================] - 0s - loss: 0.4278 - acc: 0.8430 - val_loss: 0.9463 - val_acc: 0.7869
Epoch 74/500


242/242 [==============================] - 0s - loss: 0.2508 - acc: 0.9256 - val_loss: 0.8786 - val_acc: 0.8033
Epoch 132/500
242/242 [==============================] - 0s - loss: 0.2786 - acc: 0.9008 - val_loss: 0.8837 - val_acc: 0.8033
Epoch 133/500
242/242 [==============================] - 0s - loss: 0.2364 - acc: 0.9256 - val_loss: 0.8795 - val_acc: 0.8033
Epoch 134/500
242/242 [==============================] - 0s - loss: 0.2598 - acc: 0.8884 - val_loss: 0.8725 - val_acc: 0.8197
Epoch 135/500
242/242 [==============================] - 0s - loss: 0.3028 - acc: 0.8843 - val_loss: 0.8605 - val_acc: 0.8033
Epoch 136/500
242/242 [==============================] - 0s - loss: 0.2289 - acc: 0.9174 - val_loss: 0.8860 - val_acc: 0.8033
Epoch 137/500
242/242 [==============================] - 0s - loss: 0.3300 - acc: 0.8760 - val_loss: 0.8851 - val_acc: 0.8033
Epoch 138/500
242/242 [==============================] - 0s - loss: 0.2637 - acc: 0.8884 - val_loss: 0.9041 - val_acc: 0.7541
Epoch 

242/242 [==============================] - 0s - loss: 0.1810 - acc: 0.9421 - val_loss: 0.9853 - val_acc: 0.7541
Epoch 197/500
242/242 [==============================] - 0s - loss: 0.1746 - acc: 0.9504 - val_loss: 0.9928 - val_acc: 0.7541
Epoch 198/500
242/242 [==============================] - 0s - loss: 0.2129 - acc: 0.9298 - val_loss: 1.0291 - val_acc: 0.7705
Epoch 199/500
242/242 [==============================] - 0s - loss: 0.2163 - acc: 0.9132 - val_loss: 0.9649 - val_acc: 0.7869
Epoch 200/500
242/242 [==============================] - 0s - loss: 0.2260 - acc: 0.9256 - val_loss: 0.9303 - val_acc: 0.7869
Epoch 201/500
242/242 [==============================] - 0s - loss: 0.1848 - acc: 0.9339 - val_loss: 0.9683 - val_acc: 0.7705
Epoch 202/500
242/242 [==============================] - 0s - loss: 0.1755 - acc: 0.9339 - val_loss: 1.0188 - val_acc: 0.7541
Epoch 203/500
242/242 [==============================] - 0s - loss: 0.1774 - acc: 0.9215 - val_loss: 1.0487 - val_acc: 0.7541
Epoch 

242/242 [==============================] - 0s - loss: 0.1414 - acc: 0.9463 - val_loss: 1.0791 - val_acc: 0.7869
Epoch 262/500
242/242 [==============================] - 0s - loss: 0.1527 - acc: 0.9380 - val_loss: 1.0670 - val_acc: 0.8033
Epoch 263/500
242/242 [==============================] - 0s - loss: 0.1599 - acc: 0.9504 - val_loss: 1.0721 - val_acc: 0.8033
Epoch 264/500
242/242 [==============================] - 0s - loss: 0.1263 - acc: 0.9669 - val_loss: 1.1440 - val_acc: 0.7705
Epoch 265/500
242/242 [==============================] - 0s - loss: 0.1434 - acc: 0.9463 - val_loss: 1.1635 - val_acc: 0.7377
Epoch 266/500
242/242 [==============================] - 0s - loss: 0.1534 - acc: 0.9380 - val_loss: 1.1157 - val_acc: 0.8033
Epoch 267/500
242/242 [==============================] - 0s - loss: 0.1234 - acc: 0.9711 - val_loss: 1.0598 - val_acc: 0.8033
Epoch 268/500
242/242 [==============================] - 0s - loss: 0.1248 - acc: 0.9752 - val_loss: 1.0649 - val_acc: 0.8033
Epoch 

242/242 [==============================] - 0s - loss: 0.1439 - acc: 0.9504 - val_loss: 1.0232 - val_acc: 0.8033
Epoch 327/500
242/242 [==============================] - 0s - loss: 0.1499 - acc: 0.9504 - val_loss: 1.0644 - val_acc: 0.8033
Epoch 328/500
242/242 [==============================] - 0s - loss: 0.1182 - acc: 0.9628 - val_loss: 1.1363 - val_acc: 0.8033
Epoch 329/500
242/242 [==============================] - 0s - loss: 0.0882 - acc: 0.9835 - val_loss: 1.1649 - val_acc: 0.8197
Epoch 330/500
242/242 [==============================] - 0s - loss: 0.1022 - acc: 0.9752 - val_loss: 1.1501 - val_acc: 0.7705
Epoch 331/500
242/242 [==============================] - 0s - loss: 0.1003 - acc: 0.9669 - val_loss: 1.1444 - val_acc: 0.7705
Epoch 332/500
242/242 [==============================] - 0s - loss: 0.0957 - acc: 0.9669 - val_loss: 1.1526 - val_acc: 0.8033
Epoch 333/500
242/242 [==============================] - 0s - loss: 0.1434 - acc: 0.9587 - val_loss: 1.2014 - val_acc: 0.7869
Epoch 

242/242 [==============================] - 0s - loss: 0.0721 - acc: 0.9793 - val_loss: 1.1457 - val_acc: 0.8033
Epoch 391/500
242/242 [==============================] - 0s - loss: 0.0967 - acc: 0.9752 - val_loss: 1.1726 - val_acc: 0.8033
Epoch 392/500
242/242 [==============================] - 0s - loss: 0.1101 - acc: 0.9504 - val_loss: 1.1877 - val_acc: 0.8033
Epoch 393/500
242/242 [==============================] - 0s - loss: 0.1255 - acc: 0.9669 - val_loss: 1.1766 - val_acc: 0.7869
Epoch 394/500
242/242 [==============================] - 0s - loss: 0.0964 - acc: 0.9711 - val_loss: 1.1419 - val_acc: 0.7705
Epoch 395/500
242/242 [==============================] - 0s - loss: 0.0725 - acc: 0.9793 - val_loss: 1.1063 - val_acc: 0.7705
Epoch 396/500
242/242 [==============================] - 0s - loss: 0.1045 - acc: 0.9587 - val_loss: 1.1039 - val_acc: 0.8033
Epoch 397/500
242/242 [==============================] - 0s - loss: 0.1058 - acc: 0.9587 - val_loss: 1.1321 - val_acc: 0.8033
Epoch 

242/242 [==============================] - 0s - loss: 0.0837 - acc: 0.9793 - val_loss: 1.3403 - val_acc: 0.7377
Epoch 455/500
242/242 [==============================] - 0s - loss: 0.0577 - acc: 0.9793 - val_loss: 1.3432 - val_acc: 0.7377
Epoch 456/500
242/242 [==============================] - 0s - loss: 0.0693 - acc: 0.9752 - val_loss: 1.2992 - val_acc: 0.7869
Epoch 457/500
242/242 [==============================] - 0s - loss: 0.0764 - acc: 0.9587 - val_loss: 1.2554 - val_acc: 0.8197
Epoch 458/500
242/242 [==============================] - 0s - loss: 0.0875 - acc: 0.9711 - val_loss: 1.2353 - val_acc: 0.8197
Epoch 459/500
242/242 [==============================] - 0s - loss: 0.0846 - acc: 0.9711 - val_loss: 1.2509 - val_acc: 0.8033
Epoch 460/500
242/242 [==============================] - 0s - loss: 0.0585 - acc: 0.9835 - val_loss: 1.2964 - val_acc: 0.8033
Epoch 461/500
242/242 [==============================] - 0s - loss: 0.0731 - acc: 0.9545 - val_loss: 1.3730 - val_acc: 0.7869
Epoch 

## Open Issue 1: We have two separate models now
* How do we evaluate?
* How to save model for later prediction use / deployment?

In [32]:
from keras import models

combined_model = models.Sequential()
combined_model.add(vgg_model)
combined_model.add(classifier_model)

In [33]:
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
model_1 (Model)              (None, 6)                 526086    
Total params: 15,240,774
Trainable params: 15,240,774
Non-trainable params: 0
_________________________________________________________________


In [62]:
combined_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
train_loss, train_accuracy = combined_model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

303/303 [==============================] - 25s    


(0.27357645196956965, 0.95049505442282545)

In [36]:
test_loss, test_accuracy = combined_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

76/76 [==============================] - 5s


(0.4798566997051239, 0.86842107772827148)

In [37]:
combined_model.save('vgg16-retrained.hdf5')
# combined_model.save('vgg16-augmented-retrained.hdf5')

In [39]:
!ls -lh vgg16-retrained.hdf5
# !ls -lh vgg16-augmented-retrained.hdf5

-rwxrwxrwx 1 root root 59M Sep 30 13:53 vgg16-retrained.hdf5
-rwxrwxrwx 1 root root 59M Sep 30 13:30 vgg16-auugmented-retrained.hdf5


## Open Issue 2: Whatever we do, we overfit, much more than 80% on test not possible
* first thing we could try: maybe bottlebeck feature being 2x2 is too small, we could compensate by scaling images up to 128x128 or even 256x256
  * this can indeed bring up test score to 90%
  * however, this will make the model incompatible with the 64x64 input of the other models and make deployment harder, so we keep 64x64
* maybe feature extracting from Imagenet is too different from what we have with speed limit signs? 
* or is the classifier too simply for the complex features?

## Let us try some fine tuning

### First we freeze all but the last convolutional block

In [42]:
len(vgg_model.layers)

19

In [43]:
vgg_model.layers

In [44]:
first_conv_layer = vgg_model.layers[1]

In [46]:
first_conv_layer.trainable

True

In [48]:
# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# so, the general features are kept and we (hopefully) do not have overfitting
non_trainable_layers = vgg_model.layers[:15]

In [49]:
non_trainable_layers

In [50]:
for layer in non_trainable_layers:
    layer.trainable = False

In [51]:
first_conv_layer.trainable

False

### We then tweak the complete model by very slowly re-retraining classifier and final convolutional block
* leave everthing else in place
* slow learning prevents us from ruining previous good results
* earlier layers hopefully already encode common feaure channels
* less risk of overfitting
  * earlier layers are more general
  * model has too much capacity for training and is likley to learn each and every detail
* a little bit faster

#### This may still take quite a while

In [63]:
from keras import optimizers

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate
# make updates very small and non adaptive so we do not ruin previous learnings 
combined_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [56]:
!rm -r tf_log

In [57]:
%time combined_model.fit(X_train, y_train, epochs=100, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[tb_callback])

Train on 242 samples, validate on 61 samples
Epoch 1/100
242/242 [==============================] - 28s - loss: 0.0176 - acc: 0.9979 - val_loss: 0.3353 - val_acc: 0.9208
Epoch 2/100
242/242 [==============================] - 28s - loss: 0.0230 - acc: 0.9917 - val_loss: 0.3276 - val_acc: 0.9235
Epoch 3/100
242/242 [==============================] - 28s - loss: 0.0268 - acc: 0.9910 - val_loss: 0.3271 - val_acc: 0.9344
Epoch 4/100
242/242 [==============================] - 28s - loss: 0.0218 - acc: 0.9945 - val_loss: 0.3358 - val_acc: 0.9344
Epoch 5/100
242/242 [==============================] - 27s - loss: 0.0292 - acc: 0.9897 - val_loss: 0.3470 - val_acc: 0.9344
Epoch 6/100
242/242 [==============================] - 27s - loss: 0.0218 - acc: 0.9945 - val_loss: 0.3516 - val_acc: 0.9317
Epoch 7/100
242/242 [==============================] - 28s - loss: 0.0199 - acc: 0.9931 - val_loss: 0.3537 - val_acc: 0.9290
Epoch 8/100
242/242 [==============================] - 28s - loss: 0.0143 - acc:

242/242 [==============================] - 28s - loss: 0.0093 - acc: 0.9972 - val_loss: 0.3636 - val_acc: 0.9290
Epoch 66/100
242/242 [==============================] - 27s - loss: 0.0083 - acc: 0.9972 - val_loss: 0.3620 - val_acc: 0.9290
Epoch 67/100
242/242 [==============================] - 28s - loss: 0.0126 - acc: 0.9952 - val_loss: 0.3594 - val_acc: 0.9344
Epoch 68/100
242/242 [==============================] - 27s - loss: 0.0112 - acc: 0.9952 - val_loss: 0.3582 - val_acc: 0.9344
Epoch 69/100
242/242 [==============================] - 27s - loss: 0.0082 - acc: 0.9986 - val_loss: 0.3592 - val_acc: 0.9344
Epoch 70/100
242/242 [==============================] - 27s - loss: 0.0109 - acc: 0.9986 - val_loss: 0.3589 - val_acc: 0.9372
Epoch 71/100
242/242 [==============================] - 27s - loss: 0.0066 - acc: 0.9979 - val_loss: 0.3571 - val_acc: 0.9344
Epoch 72/100
242/242 [==============================] - 27s - loss: 0.0135 - acc: 0.9959 - val_loss: 0.3536 - val_acc: 0.9235
Epoch

In [64]:
train_loss, train_accuracy = combined_model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

303/303 [==============================] - 32s    


(0.28273354007834844, 0.96039603507951543)

In [65]:
test_loss, test_accuracy = combined_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

76/76 [==============================] - 6s


(0.5479854941368103, 0.85526317358016968)

In [66]:
combined_model.save('vgg16-retrained-fine-tuned.hdf5')

In [67]:
!ls -lh vgg16-retrained-fine-tuned.hdf5

-rwxrwxrwx 1 root root 59M Sep 30 15:49 vgg16-retrained-fine-tuned.hdf5
